앞서 \ref{chap:TyInf}장에서 TODO
\lipsum[3-3]

TODO 일단 12장 내용을 템플릿으로 복사만 해 놓은 상태 TODO

In [1]:
:opt no-lint
{-# LANGUAGE TypeSynonymInstances FlexibleInstances #-}

\newpage
# 재귀함수요약식의 타입복원이 어려운 이유
앞서 \ref{sec:recTyRecon}절에서 $\textbf{rec}\,f\,(\lambda x{:}\tau_2.e)$와 같은 재귀함수요약식에서 $f$의
타입 표기 복원이 어렵다고 언급한 바 있는데 여기서는 그 이유를 생각해 보자.
타입복원(`recon`) 함수는 타입환경(`TEnv`)과 식(`Expr`)을 인자로 받아 
타입유추(`infer`) 함수를 활용해 부분식으로부터 복원에 필요한 타입 정보를 얻어가며
생략된 타입 표기에 대한 복원을 진행한다.
설명의 편의를 위해 하스켈 코드와 타입규칙의 표현을 혼용한 의사코드(pseudocode)로 나타내자면
대략 다음과 같이 진행되어야 할텐데
\newline
`recon` $\Gamma$ $(\textbf{rec}\,f\,(\lambda x{:}\tau_2.e))$ = `[` $\!\!\textbf{rec}\,f{:}\tau_1\,(\lambda x{:}\tau_2.e)\!$ `|` $\!\tau_1$ $\leftarrow$ `infer` $\{x{:}\tau_2,\,f{:}??\}\Gamma$ $e\!\!$ `]` 
\newline
문제는 $??$의 위치가 과연 무슨 타입이어야 하는가이다.
$f$는 재귀함수요약식 자체를 대표하므로 당연히 $\tau_1$이면 되지 않냐고 생각했다면,
물론 맞는 생각이기는 하지만, 문제는 그런 방식으로 프로그램을 작성할 수가 없다는 것이다.
왜냐하면 $\tau_1$은 타입유추(`infer`)의 계산 결과로 비로소 알게 되는 타입인데,
`infer` 함수의 첫째 인자인 확장된 환경이 $\tau_1$을 포함한다면 이는 계산하기도 전에
결과값을 미리 알고 제공해야 한다는 이야기이므로 말이 되지 않는다.

이런 어려움을 극복하려면 말이 되지 않는 상황을 회피할 방법을 찾으면 된다.
말이 되지 않았던 이유는 $\tau_1$이라는 계산 결과를 미리 `완전히` 알고 타입환경을 $f$의 바인딩을 포함해야 한다는 생각 때문이었다.
그렇다면 불완전한 타입 정보를 타입 환경의 바인딩 내용으로 제공하거나 심지어 식에도 불완전한 타입 정보를 표기하는 것을 허용한다면 어떨까?
불완전한 정보 중에서 알려지지 않은 부분이 결국 무엇인지는 여전히 해결해야 할 숙제로 미뤄지긴 했지만, 최소한 완전히 말이 되지 않는 상황에서는 일단 벗어나서 생각해 볼 수 있다.
사실 이러한 접근 방식은 우리가 이미 수학에서 미지수를 포함한 \`방정식\'의 풀이에서 익숙한 문제 해결 방식이다.
예컨대, $x = 2\times x - 3$이라는 방정식은 미지수 $x$를 포함한 우변의 산술식 $2\times x - 3$의 계산 결과가 좌변, 즉 $x$라는 말이다.
이렇게 아직 알려지지 않은 계산 결과를 일부분으로 포함하는 방정식을 해결하는 방법이 있듯, 재귀함수요약식의 타입복원도 마찬가지로 해결할 수 있으리라는 희망을 가져 볼 수 있다.

# TODO
TODO

In [2]:
type Nm = String  -- 변수 이름은 문자열로
data Ty = TV Int | Z | Arr Ty Ty  deriving (Eq,Ord)
data Expr = Var Nm                      -- x
          | Lam Nm (Maybe Ty) Expr      -- (\x:t.e)
          | Rec Nm (Maybe Ty) Expr      -- rec f:t (\x:t2.e)
          | App Expr Expr               -- (e1 e2)
          | Lit Int                     -- n
          | Add Expr Expr               -- e1 + e2
          | If Expr Expr Expr           -- if e then e1 else e0
          deriving (Eq, Ord)
data Val = Cl Expr Env | Vi Int  deriving (Eq,Ord)
type Env = [ (Nm, Val) ]
type TEnv = [ (Nm, Ty) ]

In [3]:
instance Show Ty where
  showsPrec p (TV i) = showString("t_{"++show i++"}")
  showsPrec _ Z = showString "\\mathbb{Z}"
  showsPrec p (Arr t1 t2) = showParen (p > 1) $
      showsPrec p t1 . showString "{\\to}" . showsPrec p t2
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x mt e) = showParen (p > 1) $
      showString ("\\lambda "++x) . showsMT mt .
      showString "." . showsPrec 1 e
  showsPrec p (Rec f mt e) = showParen (p > 9) $
      showString ("\\textbf{rec}\\,"++f) . showsMT mt .
      showString "\\, " . showsPrec 2 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2
  showsPrec _ (Lit n) = shows n
  showsPrec p (Add e1 e2) = showParen (p > 6) $
      showsPrec 6 e1 . showString "+" . showsPrec 7 e2
  showsPrec p (If e e1 e0) = showParen (p > 2) $
      showString "\\textbf{if}\\;" . showsPrec 3 e .
      showString "\\;\\textbf{then}\\;" . showsPrec 3 e1 .
      showString "\\;\\textbf{else}\\;" . showsPrec 3 e0

showsMT (Just t) = showString "{:}" . showsPrec 0 t
showsMT Nothing  = id 

import IHaskell.Display (latex)
instance IHaskellDisplay Expr where -- 노트북에 디스플레이하는 인스턴스 선언
    display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Ty where
    display t = display [latex $ "$"++show t++"$"]

In [4]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show TEnv where
    show tenv = "\\{"++ intercalate ",\\," (map show tenv) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Val) where
    show (x,v) = x++"{\\mapsto}"++show v
instance {-# OVERLAPS #-} Show (Nm,Ty) where
    show (x,t) = x++"{\\mapsto}"++show t
instance Show Val where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "
    show (Vi n) = " "++show n++" "

instance {-# OVERLAPS #-} IHaskellDisplay Env where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay TEnv where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Val) where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Ty) where
    display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Val where
    display v = display [latex $ "$"++show v++"$"]

In [5]:
recon :: TEnv -> Expr -> Expr  -- 무조건 Nothing을 타입변수로
recon tenv e = rcn 0 tenv e

rcn i _    e@(Var _) = e
rcn i tenv (Lam x Nothing  e) = Lam x (Just t) (rcn (1+i) tenv' e)
                            where tenv' = (x,t):(tenv)
                                  t = TV i
rcn i tenv (Lam x (Just t) e) = Lam x (Just t) (rcn i tenv' e)
                            where tenv' = (x,t):tenv
rcn i tenv (Rec f Nothing  e) = Rec f (Just t) (rcn (1+1) tenv' e)
                            where tenv' = (f,t):(tenv)
                                  t = TV i
rcn i tenv (Rec f (Just t) e) = Rec f (Just t) (rcn (1+i) tenv' e)
                            where tenv' = (f,t):tenv
rcn i tenv (App e1 e2)        = App e1' e2'
                            where e1' = rcn i tenv e1
                                  e2' = rcn j tenv e2
                                  j = 1 + maxTV i e1'           
rcn i _    e@(Lit _) = e
rcn i tenv (Add e1 e2)        = Add e1' e2'
                            where e1' = rcn i tenv e1
                                  e2' = rcn j tenv e2
                                  j = 1 + maxTV i e1'
rcn i tenv (If e e1 e0)       = If e' e1' e0'
                            where e'  = rcn i tenv e
                                  e1' = rcn j tenv e1
                                  e0' = rcn j tenv e0
                                  j = 1 + maxTV i e'
                                  k = 1 + maxTV j e1'

maxTV :: Int -> Expr -> Int
maxTV n e = maximum (n : concatMap collectTV (collectTy e))

collectTy :: Expr -> [Ty]
collectTy (Var _) = []
collectTy (Lam x Nothing  e) = collectTy e
collectTy (Lam x (Just t) e) = t : collectTy e
collectTy (Rec x Nothing  e) = collectTy e
collectTy (Rec x (Just t) e) = t : collectTy e
collectTy (App e1 e2) = collectTy e1 ++ collectTy e2
collectTy (Lit _) = []
collectTy (Add e1 e2) = collectTy e1 ++ collectTy e2
collectTy (If e e1 e0) = collectTy e ++ collectTy e1 ++ collectTy e0

collectTV :: Ty -> [Int]
collectTV (TV i)      = [i]
collectTV Z           = []
collectTV (Arr t1 t2) = collectTV t1 ++ collectTV t2

In [6]:
e1 = Lam "x" Nothing . Lam "x'" Nothing $ Var "x" `Add` Var "x'"  
e2 = Lit 2 `Add` Var "y"  -- y는 자유변수!!!
e5 = Lam "z" Nothing (Var "z") `App` Lit 5
App (App e1 e2) e5

In [7]:
recon [("y",Z)] (App (App e1 e2) e5)

타입검사 함수 `check`와 타입유추 함수 `infer`는 기존의 내용을 사실상 그대로 재사용할 것이다.
타입 표기가 전혀 생략되지 않은 식, 즉 `Nothing`이 하나도 나타나지 않는 식에 대해서만
`check`의 결과가 참(`True`)이 될 수 있으며 `infer`의 결과도 비어있지 않고 유추된 타입이
들어있는 리스트(`[`$\tau\!\!$ `]`)가 될 수 있다.

In [8]:
check tenv e t = infer tenv e == [t]  -- check는 infer로 간단히 정의 가능

infer :: TEnv -> Expr -> [Ty]  -- 적절한 타입이 존재하지 않으면 빈 리스트
infer tenv (Var x)      = case lookup x tenv of Just t  -> [t]
                                                Nothing -> []
infer tenv (Lam x (Just t2) e) = undefined
infer tenv (Lam x Nothing   e) = []  -- 생략된 타입 표기가 있으면 실패
infer tenv (Rec f (Just t) e)  = undefined
infer tenv (Rec f Nothing  e)  = []  -- 생략된 타입 표기가 있으면 실패
infer tenv (App e1 e2)  = undefined
infer tevv (Lit _)      = [Z]
infer tenv (Add e1 e2)  = [Z | check tenv e1 Z, check tenv e2 Z]
infer tenv (If e e1 e0) = undefined

\noindent
\ref{chap:TyChk}장에서와 마찬가지 내용으로 위의 `infer`와 아래의 `ev` 함수에서
`undefined`로 처리된 부분을 작성하면 된다.

인터프리터 함수 `ev` 및 `eval'`도 기존의 내용을 그대로 재사용한다.
아래의 인터프리터 함수 `ev`는 타입유추가 성공한 경우, 즉 `infer`의
결과가 빈 리스트가 아닌 경우에만 실행한다고 가정한다.

In [9]:
ev :: Env -> Expr -> Val  -- infer가 성공한 경우에만 실행함을 가정
ev env (Var x) = v    where Just v = lookup x env
ev env e@(Lam _ _ _) = Cl e env
ev env (Rec f t e@(Lam _ _ _)) = undefined -- 적절히 작성
ev env (App e1 e2) = ev ((x,v2):env1) e
    where Cl (Lam x _ e) env1 = ev env e1 
          v2                  = ev env e2 
ev _   (Lit n)     = Vi n
ev env (Add e1 e2) = Vi (n1 + n2)  where Vi n1 = ev env e1
                                         Vi n2 = ev env e2
ev env (If e e1 e0) = if n/=0 then ev env e1 else ev env e0
    where Vi n = ev env e

In [10]:
-- infer에 성공한 닫힌식에 대한 인터프리터
eval' e = [ev [] e | t <- infer [] e]

In [11]:
e4 = Rec "f" (Just $ Arr Z Z) .
         Lam "i" (Just Z) $ If i (i `Add` App f (i `Add` Lit (-1))) i
    where f = Var "f"
          i = Var "i"
e4 -- rec f (\i.if i t hen i + f(i + -1) else i)

\noindent
0부터 주어진 자연수 $i$까지의 총합을 구하는 재귀함수 `e4`를 위와 같이 작성하여
다음과 같이 실행해 보자.
`infer`와 `ev`함수의 `undefined` 부분을 잘 채워넣었다면
아래에 보이는 에러 없이 정상적으로 0부터 100까지의 합 5050이 계산될 것이다.

In [12]:
-- rec f (\i.if i then i + f(i + -1) else i) 100
eval' (e4 `App` Lit 100) -- 100부터 0까지의 총합

: 

\section*{요점정리}
* TODO
* TODO

\section*{연습문제}
1. TODO
1. TODO

\section*{탐구과제}
1. C++과 Java는 모든 지역변수의 타입을 반드시 표기해야 하는 언어로 시작했지만
   최근에는 지역변수를 선언할 때 타입을 프로그래머가 구체적으로 표기하지 않아도 컴파일러가
   추측해 주는, 이 장에서 설명한 타입복원과` 같은 기능을 지원하고 있다.
   이러한 기능이 C++이나 Java에서 어떻게 발전했는지 조사해 보라.
1. TODO aaa TODO